## TicTacToe ES Implementation

Code for the environment, and inspiration was sourced from https://github.com/Zeta36/pytorch-es-tic-tac-toe/tree/master

### The policy

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class simpleMLP(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(simpleMLP, self).__init__()
        self.linear1 = nn.Linear(state_dim, 64)
        self.linear2 = nn.Linear(64, 32)
        self.out = nn.Linear(32, action_dim)

    def forward(self, x):
        x = nn.functional.relu(self.linear1(x))
        x = nn.functional.relu(self.linear2(x))
        o = self.out(x)
        return o
    
    def count_parameters(self):
        count = 0
        for param in self.parameters():
            count += param.data.numpy().flatten().shape[0]
        return count

    def es_params(self):
        """
        The params that should be trained by ES (all of them)
        """
        return [(k, v) for k, v in zip(self.state_dict().keys(),
                                       self.state_dict().values())]
    
class ES(torch.nn.Module):

    def __init__(self, num_inputs, num_outputs):
        """
        Really I should be using inheritance for the small_net here
        """
        super(ES, self).__init__()

        self.linear1 = nn.Linear(num_inputs, 256)
        self.linear2 = nn.Linear(256, 256)
        self.linear3 = nn.Linear(256, 256)
        self.actor_linear = nn.Linear(256, num_outputs)

        self.train()

    def forward(self, inputs):

        x = F.elu(self.linear1(inputs))
        x = F.elu(self.linear2(x))
        x = F.elu(self.linear3(x))

        return self.actor_linear(x)

    def count_parameters(self):
        count = 0
        for param in self.parameters():
            count += param.data.numpy().flatten().shape[0]
        return count

    def es_params(self):
        """
        The params that should be trained by ES (all of them)
        """
        return [(k, v) for k, v in zip(self.state_dict().keys(),
                                        self.state_dict().values())]

### The environment

In [2]:
from __future__ import absolute_import, division, print_function

import numpy as np
import random

class TicTacToeEnv:

    def __init__(self):
        self.observation_space = 9
        self.board = list('---------')
        self.action_space = 9

    def step(self, action):
        """
        Given an action, the model moves and we observe the new state and return the reward .
        Arguments:
              action: a integer between 0 and 8 (the place to move).
        Return Value:
               new state of board, reward, game ended (True|False)
        """
        if self.board[action] != '-':
            # Ilegal move
            return self.getBoard(), -0.2, True

        #Model moves
        self.board[action] = 'X'

        #Is Draw
        if self.isDraw(self.board):
            return self.getBoard(), 1, True

        #Classic AI moves
        reward, move = self.nextMove(self.board, 'O')
        self.board[move] = 'O'

        # Classic AI wins
        if self.isWin(self.board):
            return self.getBoard(), -1, True

        #Is Draw
        if self.isDraw(self.board):
            return self.getBoard(), 1, True

        return self.getBoard(), 0.1, False

    def getBoard(self):
        board = np.asarray([ord(i) for i in self.board], np.float32)
        return board

    def clearProb(self, prob):
        for i in range(9):
            if self.board[i] != '-':
                prob[i] = 0.0
        return prob

    def chunks(slef, l, n):
        return [l[i:i + n] for i in range(0, len(l), n)]

    def reset(self, first):
        """
        Start a new game where first move 'first'
        Return: the state of the reset board
        """
        self.board = list('---------')
        if first == 0:
            p = random.random()
            if p < 0.5:
                _, move = self.nextMove(self.board, 'O')
                self.board[move] = 'O'
                print("\nClassical AI moves first with 'O':\n")
                return self.getBoard()
            else:
                print("\nModel moves first with 'X':\n")
                return self.getBoard()
        elif first == 1:
            return self.getBoard()
        else:
            _, move = self.nextMove(self.board, 'O')
            self.board[move] = 'O'
            return self.getBoard()

    def render(self):
        """
        Print the board state
        """
        board2D = np.array(self.chunks(self.board, 3))
        for i in range(3):
                print(board2D[i].flatten())

        print('\nX = Model AI, O = Classical AI\n')

    def isDraw(self, board):
        """
        Was the game a draw?        
        Return: True|False
        """
        for i in range(9):
            if board[i] == '-':
                return False
        return True

    # Taken from https://gist.github.com/SudhagarS/3942029
    def isWin(self, board):
        """
        Given a board checks if it is in a winning state.
        Arguments:
              board: a list containing X,O or -.
        Return Value:
               True if board in winning state. Else False
        """
        ### check if any of the rows has winning combination
        for i in range(3):
            if len(set(board[i * 3:i * 3 + 3])) is 1 and board[i * 3] is not '-': return True
        ### check if any of the Columns has winning combination
        for i in range(3):
            if (board[i] is board[i + 3]) and (board[i] is board[i + 6]) and board[i] is not '-':
                return True
        ### 2,4,6 and 0,4,8 cases
        if board[0] is board[4] and board[4] is board[8] and board[4] is not '-':
            return True
        if board[2] is board[4] and board[4] is board[6] and board[4] is not '-':
            return True
        return False

    def nextMove(self, board, player):
        """
        Computes the next move for a player given the current board state and also
        computes if the player will win or not.
        Arguments:
            board: list containing X,- and O
            player: one character string 'X' or 'O'
        Return Value:
            willwin: 1 if 'X' is in winning state, 0 if the game is draw and -1 if 'O' is
                        winning
            nextmove: position where the player can play the next move so that the
                             player wins or draws or delays the loss
        """
        ### when board is '---------' evaluating next move takes some time since
        ### the tree has 9! nodes. But it is clear in that state, the result is a draw
        if len(set(board)) == 1: return 0, 4

        nextplayer = 'X' if player == 'O' else 'O'
        if self.isWin(board):
            if player is 'X':
                return -1, -1
            else:
                return 1, -1
        res_list = []  # list for appending the result
        c = board.count('-')
        if c is 0:
            return 0, -1
        _list = []  # list for storing the indexes where '-' appears
        for i in range(len(board)):
            if board[i] == '-':
                _list.append(i)
        # tempboardlist=list(board)
        for i in _list:
            board[i] = player
            ret, move = self.nextMove(board, nextplayer)
            res_list.append(ret)
            board[i] = '-'
        if player is 'X':
            maxele = max(res_list)
            return maxele, _list[res_list.index(maxele)]
        else:
            minele = min(res_list)
            return minele, _list[res_list.index(minele)]

<>:113: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:113: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:116: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:119: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:121: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:144: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:150: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:162: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:113: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:113: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:116: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:119: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:121: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:144: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:150: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:162: SyntaxWarning: 

## ES Algorithm

The components we need are:

* Function to add noise to the weights of the model
    * **Input**: Neural network/actor
    * **Output**: Array (population) of weights with noise added 
    * Key part of ES, noise is added to model weights to create a population (represents exploration)

* Function to evaluate an agent by running it in the environment and computing total reward
    * **Input**: Neural network/actor , environment
    * **Output**: Total reward, calculated over certain episode length 
    * Finds the action, based on the NN 
    * Determines reward and a new observation state based on stepping through the environment
    * Sums the reward after each step into a variable (total_reward)

* Function to evaluate the "noisy" agent by adding noise to the "plain" agent and then using the prior (^) function
    * **Input**: Noise generated by noise function , neural network , environment
    * **Output**: Reward for specific "noisy" NN

* Rollout/worker function
    * **Input**: params_queue (a global unit to store params) , output_queue (a global unit to store outputs)
    * As we are working for multiple processes
        * Each worker gets the agent NN, the sample noise, and evalutates the agent (adding the noise)
    * The env is created
    * The actor is determined (NN is created)
    * Then loop, and if the params_queue is not empty
        * Load the model with params from params_queue
        * Get a new random seed and set the new seed
        * Get noise from the noise function
        * Calculate reward from prior (^) function
        * Add this reward to output_queue (and the seed?)



In [3]:
import scipy.stats as ss


def generate_noise(neural_net):
    nn_noise = []
    for n in neural_net.parameters():
        noise = np.random.normal(size=n.data.numpy().shape)
        nn_noise.append(noise)
    return np.array(nn_noise)


def evaluate_NN(nn, env):
    obs = env.reset(1)
    game_reward = 0

    while True:
        # neural net output
        net_output = nn(torch.tensor(obs))
        # get action with max liklihood
        action = net_output.data.cpu().numpy().argmax()
        new_obs, reward, done = env.step(action)
        obs = new_obs

        game_reward += reward

        if done:
            break
    
    return game_reward

def evaluate_noisy_NN(noise, nn, env):
    print("noisy going")
    
    # save noise free params
    old_dict = nn.state_dict()

    # add noise to params
    for n, p in zip(noise, nn.parameters()):
        p.data += torch.FloatTensor(n * NOISE_STD)

    reward = evaluate_NN(nn, env)

    # load previous parameters (with no noise)
    nn.load_state_dict(old_dict)

    return reward

def worker(param_queue, output_queue):
    print("worker started")

    env = TicTacToeEnv()
    actor = simpleMLP(env.observation_space, env.action_space)
    
    while True:
        actor_params = param_queue.get()
        print("HELLO")
        print(f"Actor params: {actor_params}")
        if actor_params != None:
            print("Worker loading parameters...")
            # load actor params from queue
            actor.load_state_dict(actor_params)

            # get random seed
            seed = np.random.randint(1e5)
            
            # set new seed
            np.random.seed(seed)
            print(f"Worker using seed {seed}.")

            noise = generate_noise(actor)

            reward = evaluate_noisy_NN(noise, actor, env)

            print(f"Worker completed evaluation with reward {reward}.")

            output_queue.put([reward, seed])
        else:
            break

def normalized_rank(rewards):
    ranked = ss.rankdata(rewards)
    norm = (ranked - 1) / (len(ranked) - 1)
    norm -= 0.5
    return norm

In [4]:
import torch.multiprocessing as mp
from torch import optim
import time

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("Training on:", device)

NOISE_STD = 0.05
POPULATIONSIZE = 10
LEARNINGRATE = 0.01
MAXITERATIONS = 100

WORKERS = 1

if __name__ == '__main__':
    env = TicTacToeEnv()

    actor = simpleMLP(env.observation_space, env.action_space)
    optimizer = optim.Adam(actor.parameters(), lr = LEARNINGRATE)

    # create queues so I can pass variables between processes
    output_queue = mp.Queue(maxsize=POPULATIONSIZE)
    param_queue = mp.Queue(maxsize=POPULATIONSIZE)

    processes = []

    for _ in range(WORKERS):
        p = mp.Process(target=worker, args=(param_queue, output_queue))
        p.start()
        processes.append(p)

    print("workers started!")

    for iteration in range(MAXITERATIONS):
        it_time = time.time()

        batch_noise = []
        batch_reward = []


        print(f"Iteration {iteration} - collecting rewards...")
        # get results from each worker
        for _ in range(POPULATIONSIZE):
            r, s = output_queue.get()

            print(f"Received reward: {r} with seed {s}")

            np.random.seed(s)
            noise = generate_noise(actor)
            batch_noise.append(noise)

            batch_reward.append(r)
        
        print("population created")

        avg_reward = np.round(np.mean(batch_reward), 2)
        print(iteration, 'Mean:',avg_reward, 'Max:', np.round(np.max(batch_reward), 2), 'Time:', np.round(time.time()-it_time, 2)) 


        th_update = []
        optimizer.zero_grad()
        # for each actor's parameter, and for each noise in the batch, update it by the reward * the noise value
        for idx, p in enumerate(actor.parameters()):
            upd_weights = np.zeros(p.data.shape)

            for n,r in zip(batch_noise, batch_reward):
                upd_weights += r*n[idx]

            upd_weights = upd_weights / (POPULATIONSIZE*NOISE_STD)
            # put the updated weight on the gradient variable so that afterwards the optimizer will use it
            p.grad = torch.FloatTensor( -upd_weights)
            th_update.append(np.mean(upd_weights))

        # Optimize the actor's NN
        optimizer.step()

    # quit processes

    for _ in range(WORKERS):
        param_queue.put(None)

    for p in processes:
        p.join()




worker started
workers started!
Iteration 0 - collecting rewards...


KeyboardInterrupt: 

<Process name='Process-40' pid=12313 parent=10013 started>
